# ResNet

ResNet 模型由 Kaiming He、Xiangyu Zhang、Shaoqing Ren 和 Jian Sun 在 [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) 一文中提出。

<div class="alert alert-info">
**摘要**：深度神经网络更难训练。我们提出了一种残差学习框架，以简化比以前使用的网络深度大得多的网络的训练。我们明确地将层重新表述为参照层输入学习残差函数，而非学习无参考的函数。我们提供了全面的经验证据，表明这些残差网络更易于优化，并且可以从大幅增加的深度中获得准确性。在 ImageNet 数据集上，我们评估了深度高达 152 层的残差网络 —— 比 VGG 网络深 8 倍，但复杂度仍然较低。这些残差网络的集成在 ImageNet 测试集上实现了 3.57% 的错误率。这一结果在 2015 年 ILSVRC 分类任务中获得了第一名。我们还展示了在具有 100 层和 1000 层的 CIFAR-10 上的分析。表征网络的深度对于许多视觉识别任务至关重要。仅仅由于我们对于深度的提升，我们在 COCO 目标检测数据集上获得了 28% 的相对改进。深度残差网络是我们提交给 ILSVRC 和 COCO 2015 竞赛的基础，在那里我们也在 ImageNet 检测、ImageNet 定位、COCO 检测和 COCO 分割任务中获得了第一名。
</div>

神经网络随着层数增加而被认为更加有效，因为增加层数可以提升模型的表现。随着网络的加深，提取的特征可以得到进一步丰富，例如 VGG16 和 VGG19。于是产生了一个疑问：“网络的学习是否也像简单地堆叠更多的层一样容易？” 为了解决这个问题中的一个障碍——梯度消失问题，采用了归一化的初始化方式和中间归一化层。

然而，一个新的问题出现了：退化问题。随着神经网络变得更深，准确度趋于饱和并快速退化。一项对比 shadow 和 deep plain 网络的实验表明，深层模型在训练和测试中表现出更高的错误率，表明在有效训练深层架构方面存在根本性的挑战。这种退化不是由于过拟合，而是因为网络变深时训练误差增加。增加的层未能逼近身份函数。

ResNet的残差连接释放了极深度网络的潜力，相较于先前架构显著提升了准确性。


<div class="wy-nav-content-img">
    <img src="assets/ResNet_resnet_archs.png" alt="34 层的ResNet架构与 VGG19 以及线性连接的结构之间的对比">
    <p>图1: 34 层的ResNet架构与 VGG19 以及线性连接的结构之间的对比</p>
</div>



## ResNet 架构

ResNet的构建块被设计为身份函数，在保留输入信息的同时进行学习。这种方法确保了高效的权重优化，并防止了网络加深时的退化。

ResNet的构建模块如图所示，来源：ResNet论文。

<div class="wy-nav-content-img">
    <img src="assets/ResNet_resnet_building_block.png" width=400px alt="ResNet 架构中的残差块">
    <p>图2: ResNet 架构中的残差块</p>
</div>



快捷连接执行身份映射，其输出会加到堆叠层的输出上。身份快捷连接既不增加额外的参数也不增加计算复杂度，这些连接绕过层级，为信息流动创造了直接路径，使得神经网络能够学习残差函数 (F)。

我们可以将ResNet网络总结为 -> 平面网络 + 快捷连接！

对于操作 $F(x) + x$，$F(x)$ 和 $x$ 应具有相同的维度。 ResNet采用了两种技术来实现这一点：

* 零填充快捷连接：添加全为零的通道来保持维度，同时不引入额外需要学习的参数。
* 投影快捷连接：使用1x1卷积在必要时调整维度，包含一些可学习的额外参数。

在更深的ResNet架构中，例如ResNet 50、101 和 152，采用了一种专门的“瓶颈构建块”来管理参数复杂性，保持效率，同时允许更深度的学习。



## ResNet 的从零实现

In [1]:
from timm.models import ResNet
from transformers import ResNetModel